In [ ]:
from __future__ import annotations

import os
import json
from typing import Any, Dict, List
from dotenv import load_dotenv

import requests

LAW_SEARCH_URL = "http://www.law.go.kr/DRF/lawSearch.do"


load_dotenv()

def require_oc() -> str:
    oc = os.getenv("LAW_OC", "").strip()
    if not oc:
        raise RuntimeError("환경변수 LAW_OC가 비어있습니다. 예: LAW_OC=g4c")
    return oc


def get_json(url: str, params: Dict[str, Any], timeout: int = 30) -> Dict[str, Any]:
    r = requests.get(url, params=params, timeout=timeout)
    r.raise_for_status()
    try:
        return r.json()
    except Exception:
        head = (r.text or "")[:800]
        raise RuntimeError(f"JSON 파싱 실패. 응답 앞부분:\n{head}")


def as_list(x: Any) -> List[Any]:
    if x is None:
        return []
    return x if isinstance(x, list) else [x]


def pick_law_rows(search_resp: Dict[str, Any]) -> List[Dict[str, Any]]:
    # 흔한 형태: {"LawSearch": {"law": [...]}}
    if isinstance(search_resp.get("LawSearch"), dict) and "law" in search_resp["LawSearch"]:
        return [x for x in as_list(search_resp["LawSearch"]["law"]) if isinstance(x, dict)]
    # 다른 형태: {"law": [...]}
    if "law" in search_resp:
        return [x for x in as_list(search_resp["law"]) if isinstance(x, dict)]
    return []


def norm(v: Any) -> str:
    return str(v).strip() if v is not None else ""


def is_statute_law(row: Dict[str, Any]) -> bool:
    """법률만 남기고 시행령/시행규칙은 제외"""
    name = norm(row.get("법령명한글") or row.get("lawName"))
    kind = norm(row.get("법령구분명") or row.get("lawType"))
    if "시행령" in name or "시행규칙" in name:
        return False
    return kind == "법률"


def print_rows(title: str, rows: List[Dict[str, Any]], limit: int = 50) -> None:
    print("\n" + "=" * 90)
    print(title)
    print("=" * 90)

    if not rows:
        print("(없음)")
        return

    for i, r in enumerate(rows[:limit], start=1):
        mst = norm(r.get("법령일련번호") or r.get("MST") or r.get("mst") or r.get("lsiSeq"))
        lid = norm(r.get("법령ID") or r.get("LID") or r.get("lawId"))
        name = norm(r.get("법령명한글") or r.get("lawName"))
        kind = norm(r.get("법령구분명") or r.get("lawType"))
        efyd = norm(r.get("시행일자") or r.get("efYd"))
        anc_date = norm(r.get("공포일자") or r.get("ancYd"))
        anc_no = norm(r.get("공포번호") or r.get("ancNo"))
        org = norm(r.get("소관부처명") or r.get("orgName"))
        amend = norm(r.get("제개정구분명") or r.get("rrClsNm"))

        print(f"[{i}] {name}")
        print(f"    - 법령구분명: {kind} | 제개정: {amend}")
        print(f"    - MST(법령일련번호): {mst} | LID(법령ID): {lid}")
        print(f"    - 시행일자: {efyd} | 공포일자: {anc_date} | 공포번호: {anc_no}")
        print(f"    - 소관부처명: {org}")


def main():
    oc = require_oc()

    QUERY = "주택임대차보호법"
    EFYD = "20260102"  # 원하는 시행일자(정확히 그 날만 보고 싶으면 아래 efYd_range 사용)

    # ✅ 한 날짜만 딱 보고 싶으면 같은 값 범위로 주면 됨: "20260102~20260102"
    efYd_range = f"{EFYD}~{EFYD}"

    params = {
        "OC": oc,
        "target": "eflaw",
        "type": "JSON",
        "search": 1,            # 1: 법령명 검색
        "query": QUERY,
        "nw": 3,                # 3: 현행
        "sort": "efdes",        # 시행일자 내림차순
        "display": 100,
        "page": 1,
        "efYd": efYd_range,     # ← 한 날짜만 보고 싶지 않으면 이 줄을 주석 처리
    }

    data = get_json(LAW_SEARCH_URL, params=params)
    rows = pick_law_rows(data)

    # 전체 결과
    print(f"[INFO] total rows fetched: {len(rows)}")
    # 필요하면 원본 구조 확인
    # print(json.dumps(data, ensure_ascii=False, indent=2)[:2000])

    # 필터링: 법률만
    statutes = [r for r in rows if is_statute_law(r)]
    others = [r for r in rows if r not in statutes]

    print_rows("✅ 법률만(시행령/시행규칙 제외)", statutes, limit=50)
    print_rows("참고: 제외된 나머지(시행령/시행규칙 등)", others, limit=50)

    # 다음 단계에서 쓰게 될 핵심 값만 요약 출력
    print("\n" + "-" * 90)
    if statutes:
        best = statutes[0]
        best_mst = norm(best.get("법령일련번호"))
        best_lid = norm(best.get("법령ID"))
        best_name = norm(best.get("법령명한글"))
        best_efyd = norm(best.get("시행일자"))
        print("[NEXT STEP] 본문 호출에 사용할 후보(첫 번째 법률 row)")
        print(f"  - 법령명: {best_name}")
        print(f"  - MST: {best_mst}")
        print(f"  - LID: {best_lid}")
        print(f"  - 시행일자: {best_efyd}")
    else:
        print("[WARN] 법률 row를 찾지 못했습니다. (efYd 범위를 빼고 다시 시도해보세요)")
    print("-" * 90)


if __name__ == "__main__":
    main()